<a href="https://colab.research.google.com/github/VeronicaGomes/Hello-world/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gradio openrouteservice


In [4]:
import gradio as gr
import openrouteservice
import random
import os

# Load your API key securely from environment variables
client = openrouteservice.Client(key="5b3ce3597851110001cf6248fc64339c5c364620a8c60547bffeda31")


# Function to calculate fare and arrival time
def calculate_fares(distance_km):
    return {
        "Uber": {
            "price": round(5 + 1.2 * distance_km, 2),
            "arrival": random.randint(4, 6),
            "logo": "🚗"
        },
        "DiDi": {
            "price": round(4 + 1.15 * distance_km, 2),
            "arrival": random.randint(2, 4),
            "logo": "🚘"
        },
        "13CABS": {
            "price": round(6 + 1.1 * distance_km, 2),
            "arrival": random.randint(1, 3),
            "logo": "🚕"
        }
    }

# Function to render fare cards in HTML
def render_ui(distance_km, fares_sorted):
    html = f"<h3>📍 Route Distance: {distance_km} km</h3><div style='margin-top:10px;'>"
    for platform, details in fares_sorted:
        color = "#f9f9f9"
        if platform == "13CABS":
            color = "#fff9f9"
        elif platform == "DiDi":
            color = "#f9f9ff"
        html += f"""
        <div style='padding:15px; margin-bottom:10px; border-radius:12px; border:1px solid #ccc; background:{color};color:#222;'>
            <b>{details['logo']} {platform}</b><br>
           <span style='color:#555;'> Arrival: {details['arrival']} mins<br>
            <b style='font-size:20px;color:#000;'>${details['price']}</b>
        </div>
        """
    html += "<br><button style='padding:10px 20px; background:#FF4D00; color:white; border:none; border-radius:10px;'>Book</button></div>"
    return html

# Function to fetch route and fare estimates
def get_estimates(pickup, dropoff):
    try:
        geocode_start = client.pelias_search(text=pickup)
        geocode_end = client.pelias_search(text=dropoff)

        if not geocode_start['features'] or not geocode_end['features']:
            return "<b style='color:red;'>Invalid pickup or drop-off address.</b>", None

        coords = [
            geocode_start['features'][0]['geometry']['coordinates'],
            geocode_end['features'][0]['geometry']['coordinates']
        ]

        route = client.directions(coords)
        distance_km = round(route['routes'][0]['summary']['distance'] / 1000, 2)

        fares = calculate_fares(distance_km)
        sorted_fares = sorted(fares.items(), key=lambda x: x[1]["price"])

        return render_ui(distance_km, sorted_fares), (distance_km, fares)

    except Exception as e:
        return f"<b style='color:red;'>Error: {str(e)}</b>", None

# Sort function for price or time
def sort_fares(data, method):
    if data is None:
        return "<b style='color:red;'>Please run an estimate first.</b>"

    distance_km, fares = data
    if method == "price":
        sorted_fares = sorted(fares.items(), key=lambda x: x[1]["price"])
    elif method == "time":
        sorted_fares = sorted(fares.items(), key=lambda x: x[1]["arrival"])
    else:
        sorted_fares = fares.items()

    return render_ui(distance_km, sorted_fares)

# Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("## 🚖 FindMyRide")
    gr.Markdown("Enter pickup and drop-off locations to estimate ride fares.")

    with gr.Row():
        pickup = gr.Textbox(label="Pickup Address", placeholder="e.g. 234 Flinders Ln, Melbourne VIC")
        dropoff = gr.Textbox(label="Drop-off Address", placeholder="e.g. 1 Coora Rd, Oakleigh South VIC")

    output = gr.HTML()
    fare_data = gr.State()

    with gr.Row():
        btn_estimate = gr.Button("Estimate Fare")
        btn_sort_price = gr.Button("Sort by Price")
        btn_sort_time = gr.Button("Sort by Time")

    btn_estimate.click(fn=get_estimates, inputs=[pickup, dropoff], outputs=[output, fare_data])
    btn_sort_price.click(fn=lambda data: sort_fares(data, "price"), inputs=fare_data, outputs=output)
    btn_sort_time.click(fn=lambda data: sort_fares(data, "time"), inputs=fare_data, outputs=output)

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9e5d89f85377a7305e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
